In [1]:
from mpl_toolkits import mplot3d
from neo.core import SpikeTrain
from tqdm import tqdm
import elephant as eph
import imageio
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import quantities as q
import re 
import seaborn as sns
mpl.rcParams.update({'font.size': 20})
%config Completer.use_jedi = False

In [4]:
def eval_spikes(graphno):
    mat = np.loadtxt(f'../modules/networks/matrix_{graphno}.csv',delimiter=",")
    module = np.loadtxt(f'../modules/networks/matrix_{graphno}_modules.csv')
    order = np.argsort(module)

    all_events = []
    for pertseed in [59428,13674,84932,72957,85036]:
        events = np.load(f"../Fig 2/Data/LN30_events_{graphno}_{pertseed}.npy",allow_pickle=True)
        all_events.append(events)
    all_events = np.array(all_events,dtype=object)

    axs = []
    for rep in range(5):
        spike_times = []
        for i in range(30):
            spike_times.append(SpikeTrain(np.concatenate([all_events[0,rep,i],
                                               all_events[1,rep,i]+7000,
                                               all_events[2,rep,i]+14000,
                                               all_events[3,rep,i]+21000,
                                               all_events[4,rep,i]+28000])*q.ms,t_stop=35000*q.ms))
        spike_times = np.array(spike_times)
        bst = eph.conversion.BinnedSpikeTrain(list(spike_times),bin_size=50*q.ms)
        axes = []
        for i in np.unique(module):
            temp = np.mean(bst.to_array()[np.arange(30)[module==i],:],axis=0)
            axes.append(temp)
        axs.append(axes)
    axs = np.array(axs) 
    return axs
def order_by_variability(axs):
    axs = axs[:,np.argsort(axs.std(axis=(0,2)))[::-1],:]
    return axs, [f"M{mod+1}" for mod in np.argsort(axs.std(axis=(0,2)))]

In [11]:
def animate3d(data,graphno,nreps=5,labels=["M1","M2","M3"]):
    for rep in range(nreps):
        print(f"Replicate {rep+1}")
        lims = axs[rep][0].min(),axs[rep][0].max(),axs[rep][1].min(),axs[rep][1].max(),axs[rep][2].min(),axs[rep][2].max()
        fig = plt.figure(figsize=(5,7))
        G = gridspec.GridSpec(5, 1)

        ax1 = fig.add_subplot(G[0:4], projection='3d')
        ax1.text(-0.1, -0.1, -0.1, "0", color='k',fontdict={"fontsize":10})
        ax1.quiver(0,0,0,lims[1],0,0,arrow_length_ratio=0.1,color='k')
        ax1.text(lims[1]+0.05, -0.05, -0.05, labels[0] , color='k',fontdict={"fontsize":10})
        ax1.quiver(0,0,0,0,lims[3],0,arrow_length_ratio=0.1,color='k')
        ax1.text(-0.05,lims[3]+0.05,-0.05, labels[1], color='k',fontdict={"fontsize":10})
        ax1.quiver(0,0,0,0,0,lims[5],arrow_length_ratio=0.1,color='k')
        ax1.text(-0.05,-0.05,lims[5]+0.05, labels[2], color='k',fontdict={"fontsize":10})
        ax1.set_xlim(lims[0],lims[1])
        ax1.set_ylim(lims[2],lims[3])
        ax1.set_zlim(lims[4],lims[5])
        ax1.set_axis_off()
        ax1.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
        ax1.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
        ax1.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
        ax1.xaxis._axinfo["grid"]['color'] =  (1,1,1,0)
        ax1.yaxis._axinfo["grid"]['color'] =  (1,1,1,0)
        ax1.zaxis._axinfo["grid"]['color'] =  (1,1,1,0)

        line = ax1.plot3D(0, 0, 0, 'gray',linewidth=1)
        dot = ax1.plot3D(0, 0, 0,marker='o',color='k')

        tim = ax1.text2D(0.4,0.05,"t=0.0s",transform=ax1.transAxes,fontsize=10)

        ax2 = fig.add_subplot(G[4])
        tracks = []
        endpoint = []
        for i in range(axs.shape[1]):
            a = ax2.plot(np.arange(0,0.05*1,0.05),axs[rep][0][0],"-",color=plt.cm.inferno(i/axs.shape[1]),linewidth=0.5,label=f'M{i+1}')
            tracks.append(a)
            a = ax2.plot(0,0,"o",color=plt.cm.inferno(i/axs.shape[1]))
            endpoint.append(a)
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.05),
                  ncol=axs.shape[1], fontsize=10, frameon=False)
        ax2.set_xlim((0,35))
        ax2.set_ylim((0,1.2*np.max([lims[1],lims[3],lims[5]])))
        ax2.set_xlabel("Time (in s)")
        ax2.set_ylabel("Mean Activity")

        perts = []
        for i in range(5):
            for j in range(5):
                perts.append(7*i+1.5+j)
        pert_counter = 0

        plt.subplots_adjust(wspace=0.0,hspace=0.0)

        for i in tqdm(range(10,axs.shape[2])):
            tim.set_text(f"t = {i/20:0.3f}s")
            tim.set_color("black")
            if pert_counter<len(perts) and perts[pert_counter]<i/20:
                ax2.fill_betweenx((0,1.2*np.max([lims[1],lims[3],lims[5]])),perts[pert_counter],perts[pert_counter]+0.1,color='k',alpha=0.1)
                pert_counter+=1
                tim.set_text(f"t = {i/20:0.3f} s")
                tim.set_color("red")
            xline = axs[rep][0][i-10:i]
            yline = axs[rep][1][i-10:i]
            zline = axs[rep][2][i-10:i]
            line[0].set_data_3d(xline, yline, zline)
            dot[0].set_data_3d(xline[-1], yline[-1], zline[-1])
            plt.gca().spines['top'].set_visible(False)
            plt.gca().spines['right'].set_visible(False)
            ax1.view_init(elev=20, azim=i*0.3)
            for j in range(axs.shape[1]):
                tracks[j][0].set_xdata(np.arange(i)/20)
                tracks[j][0].set_ydata(axs[rep][j][:i])
                endpoint[j][0].set_xdata(i/20)
                endpoint[j][0].set_ydata(axs[rep][j][i])
            plt.savefig(f"anim/{i}.png", dpi=90)
        plt.close()
        
        images = []
        files = os.listdir('anim')
        files.sort(key=lambda f: int(re.sub('\D', '', f)))
        for filename in tqdm(files):
            images.append(imageio.imread('anim/'+filename))
        imageio.mimsave(f'Videos/movie_{graphno}_{rep}.gif', images,fps=20)
        
        for file in os.listdir('anim/'):
            os.remove('anim/'+file)

def animate2d(data,graphno,nreps=5,labels=["M1","M2"]):
    for rep in range(nreps):
        print(f"Replicate {rep+1}")
        lims = axs[rep][0].min(),axs[rep][0].max(),axs[rep][1].min(),axs[rep][1].max()
        fig = plt.figure(figsize=(5,7))
        G = gridspec.GridSpec(5, 1)

        ax1 = fig.add_subplot(G[0:4])
        ax1.set_xlim(lims[0],lims[1])
        ax1.set_ylim(lims[2],lims[3])
        ax1.set_xticks([lims[0],lims[1]])
        ax1.set_xticklabels(["0",labels[0]])
        ax1.set_yticks([lims[2],lims[3]])
        ax1.set_yticklabels(["",labels[1]])
        line = ax1.plot(0, 0, 'gray',linewidth=1)
        dot = ax1.plot(0, 0, marker='o',color='k')

        tim = ax1.text(0.4,0.05,"t=0.0s",transform=ax1.transAxes,fontsize=10)
        ax1.spines['right'].set_visible(False)
        ax1.spines['top'].set_visible(False)

        ax2 = fig.add_subplot(G[4])
        tracks = []
        endpoint = []
        for i in range(axs.shape[1]):
            a = ax2.plot(np.arange(0,0.05*1,0.05),axs[rep][0][0],"-",color=plt.cm.inferno(i/axs.shape[1]),linewidth=0.5,label=f'M{i+1}')
            tracks.append(a)
            a = ax2.plot(0,0,"o",color=plt.cm.inferno(i/axs.shape[1]))
            endpoint.append(a)
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.05),
                  ncol=3, fontsize=10, frameon=False)
        ax2.set_xlim((0,35))
        ax2.set_ylim((0,1.2*np.max([lims[1],lims[3]])))
        ax2.set_xlabel("Time (in s)")
        ax2.set_ylabel("Mean Activity")

        perts = []
        for i in range(5):
            for j in range(5):
                perts.append(7*i+1.5+j)
        pert_counter = 0

        plt.subplots_adjust(wspace=0.0,hspace=0.5)

        for i in tqdm(range(10,axs.shape[2])):
            tim.set_text(f"t = {i/20:0.3f}s")
            tim.set_color("black")
            if pert_counter<len(perts) and perts[pert_counter]<i/20:
                ax2.fill_betweenx((0,1.2*np.max([lims[1],lims[3]])),perts[pert_counter],perts[pert_counter]+0.1,color='k',alpha=0.1)
                pert_counter+=1
                tim.set_text(f"t = {i/20:0.3f} s")
                tim.set_color("red")
            xline = axs[rep][0][i-10:i]
            yline = axs[rep][1][i-10:i]
            line[0].set_data(xline, yline)
            dot[0].set_data(xline[-1], yline[-1])
            plt.gca().spines['top'].set_visible(False)
            plt.gca().spines['right'].set_visible(False)
            for j in range(axs.shape[1]):
                tracks[j][0].set_xdata(np.arange(i)/20)
                tracks[j][0].set_ydata(axs[rep][j][:i])
                endpoint[j][0].set_xdata(i/20)
                endpoint[j][0].set_ydata(axs[rep][j][i])
            plt.savefig(f"anim/{i}.png", dpi=90)
        plt.close()
        
        images = []
        files = os.listdir('anim')
        files.sort(key=lambda f: int(re.sub('\D', '', f)))
        for filename in tqdm(files):
            images.append(imageio.imread('anim/'+filename))
        imageio.mimsave(f'Videos/movie_{graphno}_{rep}.gif', images,fps=20)
        
        for file in os.listdir('anim/'):
            os.remove('anim/'+file)

In [15]:
axs = eval_spikes(1)
print(axs.shape)
axs,labels = order_by_variability(axs)
animate3d(axs,1,labels=labels)

<ipython-input-4-434cf04e5393>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  spike_times = np.array(spike_times)
  0%|                                                                                  | 1/690 [00:00<01:48,  6.38it/s]

(5, 4, 700)
Replicate 1


  0%|                                                                                          | 0/690 [00:00<?, ?it/s]

Replicate 2


  0%|                                                                                  | 1/690 [00:00<01:47,  6.40it/s]

...

  0%|                                                                                  | 1/690 [00:00<01:49,  6.30it/s]

Replicate 4


  0%|                                                                                          | 0/690 [00:00<?, ?it/s]

Replicate 5


100%|███████████████████████████████████████████████████████████████████████████████| 690/690 [00:04<00:00, 153.89it/s]


In [12]:
axs = eval_spikes(2)
print(axs.shape)
animate2d(axs,2)

<ipython-input-4-434cf04e5393>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  spike_times = np.array(spike_times)
  0%|                                                                                  | 1/690 [00:00<01:18,  8.80it/s]

(5, 2, 700)
Replicate 1


  0%|                                                                                  | 1/690 [00:00<01:21,  8.50it/s]

Replicate 2


  0%|                                                                                  | 1/690 [00:00<01:26,  7.96it/s]

...

  0%|                                                                                  | 1/690 [00:00<01:23,  8.22it/s]

Replicate 4


  0%|                                                                                  | 1/690 [00:00<01:25,  8.02it/s]

Replicate 5


100%|███████████████████████████████████████████████████████████████████████████████| 690/690 [00:03<00:00, 177.20it/s]


In [13]:
axs = eval_spikes(3)
print(axs.shape)
animate2d(axs,3)

<ipython-input-4-434cf04e5393>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  spike_times = np.array(spike_times)
  0%|                                                                                  | 1/690 [00:00<01:34,  7.26it/s]

(5, 2, 700)
Replicate 1


  0%|                                                                                  | 1/690 [00:00<01:30,  7.60it/s]

Replicate 2


  0%|                                                                                  | 1/690 [00:00<01:28,  7.80it/s]

...

  0%|                                                                                  | 1/690 [00:00<01:23,  8.22it/s]

Replicate 4


  0%|                                                                                  | 1/690 [00:00<01:30,  7.63it/s]

Replicate 5


100%|███████████████████████████████████████████████████████████████████████████████| 690/690 [00:03<00:00, 182.84it/s]


In [14]:
axs = eval_spikes(4)
print(axs.shape)
animate3d(axs,4)

<ipython-input-4-434cf04e5393>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  spike_times = np.array(spike_times)
  0%|                                                                                  | 1/690 [00:00<01:43,  6.68it/s]

(5, 3, 700)
Replicate 1


  0%|                                                                                          | 0/690 [00:00<?, ?it/s]

Replicate 2


  0%|                                                                                  | 1/690 [00:00<01:39,  6.94it/s]

...

  0%|                                                                                  | 1/690 [00:00<01:40,  6.87it/s]

Replicate 4


  0%|                                                                                  | 1/690 [00:00<01:41,  6.82it/s]

Replicate 5


100%|███████████████████████████████████████████████████████████████████████████████| 690/690 [00:03<00:00, 172.95it/s]


In [16]:
axs = eval_spikes(5)
print(axs.shape)
animate3d(axs,5)

<ipython-input-4-434cf04e5393>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  spike_times = np.array(spike_times)
  0%|                                                                                  | 1/690 [00:00<01:37,  7.07it/s]

(5, 3, 700)
Replicate 1


  0%|                                                                                          | 0/690 [00:00<?, ?it/s]

Replicate 2


  0%|                                                                                  | 1/690 [00:00<01:43,  6.68it/s]

...

  0%|                                                                                          | 0/690 [00:00<?, ?it/s]

Replicate 4


  0%|                                                                                  | 1/690 [00:00<01:45,  6.55it/s]

Replicate 5


100%|███████████████████████████████████████████████████████████████████████████████| 690/690 [00:03<00:00, 187.17it/s]


In [17]:
axs = eval_spikes(6)
print(axs.shape)
animate2d(axs,6)

<ipython-input-4-434cf04e5393>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  spike_times = np.array(spike_times)
  0%|                                                                                  | 1/690 [00:00<01:22,  8.36it/s]

(5, 2, 700)
Replicate 1


  0%|                                                                                  | 1/690 [00:00<01:28,  7.77it/s]

Replicate 2


  0%|                                                                                  | 1/690 [00:00<01:25,  8.02it/s]

...

  0%|                                                                                  | 1/690 [00:00<01:27,  7.89it/s]

Replicate 4


  0%|                                                                                  | 1/690 [00:00<01:23,  8.25it/s]

Replicate 5


100%|███████████████████████████████████████████████████████████████████████████████| 690/690 [00:03<00:00, 192.55it/s]


In [18]:
axs = eval_spikes(7)
print(axs.shape)
animate3d(axs,7)

<ipython-input-4-434cf04e5393>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  spike_times = np.array(spike_times)
  0%|                                                                                          | 0/690 [00:00<?, ?it/s]

(5, 3, 700)
Replicate 1


  0%|                                                                                  | 1/690 [00:00<01:40,  6.85it/s]

Replicate 2


  0%|                                                                                  | 1/690 [00:00<01:41,  6.82it/s]

...

  0%|                                                                                  | 1/690 [00:00<01:38,  7.02it/s]

Replicate 4


  0%|                                                                                  | 1/690 [00:00<01:39,  6.91it/s]

Replicate 5


100%|███████████████████████████████████████████████████████████████████████████████| 690/690 [00:03<00:00, 187.72it/s]


In [19]:
axs = eval_spikes(8)
print(axs.shape)
animate2d(axs,8)

<ipython-input-4-434cf04e5393>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  spike_times = np.array(spike_times)
  0%|                                                                                  | 1/690 [00:00<01:30,  7.60it/s]

(5, 2, 700)
Replicate 1


  0%|                                                                                  | 1/690 [00:00<01:29,  7.71it/s]

Replicate 2


  0%|                                                                                  | 1/690 [00:00<01:22,  8.35it/s]

...

  0%|                                                                                  | 1/690 [00:00<01:28,  7.77it/s]

Replicate 4


  0%|                                                                                          | 0/690 [00:00<?, ?it/s]

Replicate 5


100%|███████████████████████████████████████████████████████████████████████████████| 690/690 [00:03<00:00, 173.04it/s]


In [20]:
axs = eval_spikes(9)
print(axs.shape)
animate3d(axs,9)

<ipython-input-4-434cf04e5393>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  spike_times = np.array(spike_times)
  0%|                                                                                  | 1/690 [00:00<01:36,  7.16it/s]

(5, 3, 700)
Replicate 1


  0%|                                                                                  | 1/690 [00:00<01:38,  6.98it/s]

Replicate 2


  0%|                                                                                  | 1/690 [00:00<01:38,  6.96it/s]

...

  0%|                                                                                  | 1/690 [00:00<01:36,  7.11it/s]

Replicate 4


  0%|                                                                                  | 1/690 [00:00<01:41,  6.77it/s]

Replicate 5


100%|███████████████████████████████████████████████████████████████████████████████| 690/690 [00:03<00:00, 182.51it/s]


In [21]:
axs = eval_spikes(10)
print(axs.shape)
animate2d(axs,10)

<ipython-input-4-434cf04e5393>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  spike_times = np.array(spike_times)
  0%|                                                                                  | 1/690 [00:00<01:19,  8.72it/s]

(5, 2, 700)
Replicate 1


  0%|                                                                                  | 1/690 [00:00<01:25,  8.02it/s]

Replicate 2


  0%|                                                                                  | 1/690 [00:00<01:26,  7.96it/s]

...

  0%|                                                                                  | 1/690 [00:00<01:22,  8.39it/s]

Replicate 4


  0%|                                                                                          | 0/690 [00:00<?, ?it/s]

Replicate 5


100%|███████████████████████████████████████████████████████████████████████████████| 690/690 [00:03<00:00, 190.33it/s]


In [27]:
for graphno in range(1,11):
    gifs = [imageio.get_reader(f'Videos/{x}') for x in filter(lambda v: f"_{graphno}_" in v,os.listdir("Videos/"))]
    new_gif = imageio.get_writer(f'Videos/{graphno}_out.gif',fps=20)
    for frame_number in tqdm(range(np.min([gif.get_length() for gif in gifs]))):
        datas = [gif.get_next_data() for gif in gifs]
        new_image = np.hstack(datas)
        new_gif.append_data(new_image)
    for gif in gifs:
        gif.close()
    new_gif.close()

100%|████████████████████████████████████████████████████████████████████████████████| 690/690 [01:45<00:00,  6.56it/s]


In [29]:
files = list(filter(lambda v: "_out" in v,os.listdir("Videos/")))
files.sort(key=lambda f: int(re.sub('\D', '', f)))
for graphno in range(1,11,2):
    gifs = [imageio.get_reader(f'Videos/{x}') for x in files[graphno-1:graphno+1]]
    new_gif = imageio.get_writer(f'Videos/{graphno}_{graphno+1}.gif',fps=20)
    for frame_number in tqdm(range(np.min([gif.get_length() for gif in gifs]))):
        datas = [gif.get_next_data() for gif in gifs]
        new_image = np.vstack(datas)
        new_gif.append_data(new_image)
    for gif in gifs:
        gif.close()
    new_gif.close()

100%|████████████████████████████████████████████████████████████████████████████████| 690/690 [03:05<00:00,  3.73it/s]
